### 1. Start an instance

In [ ]:
import utils
import time

# bootstrap
bootstrap = utils.Bootstrap("worksapce.config.json")
bootstrap.run()

biz_tags = [{"Key": "role", "Value": "proxy-node"}]

In [ ]:
aliyun_ecs_manager = utils.AliyunECSManager()
cmd_generator = utils.CommandGenerator()

aliyun_ecs_manager.clean(biz_tags)

settings = aliyun_ecs_manager.create_default_settings()
settings.set_biz_tags(biz_tags)

vm_property = aliyun_ecs_manager.create_instance(settings)

time.sleep(30)

cmd_content = "\n".join([
    cmd_generator.generate_install_docker_command(),
    cmd_generator.generate_install_proxy_command(),
    cmd_generator.generate_install_git_command(),
    cmd_generator.generate_config_github_command(),
    cmd_generator.generate_start_jupyter_command()
])

print(vm_property.instance_id)

aliyun_ecs_manager.execute_command(instance_id=vm_property.instance_id,
                                  cmd_content=cmd_content)

### 2. Get Config

In [ ]:
aliyun_ecs_manager = utils.AliyunECSManager()
vm_properties = aliyun_ecs_manager.get_instances(biz_tags=biz_tags)

proxy_setting_cmd = """
sslocal -s {server} \
-p 8000 \
-l 1080 \
-k passIt2020 \
-m aes-256-cfb \
-t 600
""".format(server=vm_properties[0].public_ip)

modify_hosts_cmd = """
modify_hosts add agent {server}
""".format(server=vm_properties[0].public_ip)

print(proxy_setting_cmd)
print(modify_hosts_cmd)

### 3. Add agent to local hosts (optional)
1) Save the following script to a file named "/opt/custom/modify_hosts.sh":
```bash
#!/bin/bash

# 函数：添加 hosts 条目
add_host() {
    # 检查是否已存在相同的主机名，如果存在则删除
    remove_host "$2"
    echo "$1 $2" | sudo tee -a /etc/hosts >/dev/null
}

# 函数：删除 hosts 条目
remove_host() {
    sudo sed -i '' "/$1/d" /etc/hosts
}

# 主程序

# 检查参数数量
if [ $# -lt 2 ]; then
    echo "Usage: $0 [add|remove] [hostname] [IP]"
    exit 1
fi

action="$1"
hostname="$2"
ip="$3"

# 根据操作调用相应函数
case "$action" in
    "add")
        add_host "$ip" "$hostname"
        echo "Added $hostname to hosts file."
        ;;
    "remove")
        remove_host "$hostname"
        echo "Removed $hostname from hosts file."
        ;;
    *)
        echo "Invalid action. Please use 'add' or 'remove'."
        exit 1
        ;;
esac

exit 0
```
3) Grant executing permission
```bash
chmod +x /opt/custom/modify_hosts.sh
```

4) Open your bashrc
```bash
vim ~/.zshrc
```

5) add alias "modify_hosts"
```bash
alias modify_hosts="/opt/custom/modify_hosts.sh"
```

6) activate
```bash
source ~/.zshrc
```

### 4. Clean up

In [ ]:
aliyun_ecs_manager = utils.AliyunECSManager()
aliyun_ecs_manager.clean(biz_tags)